# Nishika_compe  
## 中古マンション価格予測 2023冬の部

## 評価指標概要  
今回のコンペの評価方法は「MAE」  
また、目的変数は、取引価格(総額)について常用対数をとった「取引価格（総額）_log」  
<br>
※注意点  
テストデータは約50%のみで暫定スコアが計算されるため、テストデータでの過学習に注意する。  
1945年以前は「戦前」と表示されます。また、建築年が把握できないものは、空欄になっています。  
特徴量の作成のため以下のデータを使用可能

【国勢調査データ】
https://www.e-stat.go.jp/stat-search/files?page=1&layout=datalist&toukei=00200521&tstat=000001049104&cycle=0&tclass1=000001049105&stat_infid=000032143614&result_page=1&tclass2val=0

【公示地価データ】
https://nlftp.mlit.go.jp/ksj/old/datalist/old_KsjTmplt-L01.html


### MAEの特徴  
実際のデータと予測値の差の絶対値の平均  
・外れ値の影響を少なくした処理をしたデータに有効  
・xgboostを使用できない  
・代わりの関数として、「Fair関数」や「Psuedo-Huber関数」を用いる。  

## ライブラリインポート

In [1]:
import pandas as pd
import numpy as np
import glob
import os
#警告非表示
import warnings
warnings.simplefilter('ignore')
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import lightgbm as lgb
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xfeat import Pipeline,SelectNumerical,ArithmeticCombinations,aggregation
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
import requests
import optuna

# 前処理

In [2]:
##データ読み込み
# ディレクトリ変更
os.chdir("C:\\Users\\t-miy\\Documents\\self-study\\Nishika\\rent_winter\\train")

# 空のDataFrameを定義
train = pd.DataFrame()

# for分でcsvファイルを一気にデータを取り込む
#globモジュールによって、引数に指定されたパターンにマッチするファイルパス名を取得できる
for i in glob.glob("*.csv*"): #「.csv」を含むすべてのファイルを指定
    tmp_df = pd.read_csv(i)
# DataFrameを連結する
    train = pd.concat([train, tmp_df])
test = pd.read_csv('../test.csv')

#面積
train['面積（㎡）'] = train['面積（㎡）'].replace('2000㎡以上','2000')
train['面積（㎡）'] = train['面積（㎡）'].astype(float)
test['面積（㎡）'] = test['面積（㎡）'].replace('2000㎡以上','2000')
test['面積（㎡）'] = test['面積（㎡）'].astype(float)

#train_xは学習データ
train_x = train.drop(['取引価格（総額）_log'],axis=1)
train_y = train['取引価格（総額）_log']

#学習データとテストデータを結合する。(まとめてカテゴリ変数変換するため)
#all_df = pd.concat([train_x,test])

#target encodingを'最寄駅：名称','地区名'にかける

# trainデータに対してtarget encodingの実装
# target encodingを行うカラム名を指定
target_col = '取引価格（総額）_log'

# カテゴリカル変数のリストを指定
cat_cols = ['最寄駅：名称','地区名']

# Target Encodingを行う関数
def target_encode(df, col):
    # カテゴリ変数をグループ化して、各カテゴリの平均値を計算する
    mean = df.groupby(col)[target_col].mean()

    # カテゴリ変数をグループ化して、各カテゴリのサンプル数を計算する
    count = df.groupby(col)[target_col].count()

    # smoothingの値を指定する（過学習を防ぐためのパラメータ）
    smoothing = 1

    # 重みを計算する
    weight = 1 / (1 + np.exp(-(count - smoothing) / 1))

    # Target Encodingを計算する
    encoded = mean * weight + 0.5 * (1 - weight)

    return encoded


# カテゴリカル変数に対して欠損値を補完する
for col in cat_cols:
    # カテゴリ変数をグループ化して、各カテゴリのサンプル数を計算する
    count = train.groupby(col)[target_col].count()

    # サンプル数が一定数以下のカテゴリには、別の値を代入する
    threshold = 5
    count[count <= threshold] = np.nan

    # 欠損値を補完する
    train[col] = train[col].fillna('other')
    train[col+'_te'] = train[col].map(target_encode(train, col))

    # testデータで計算した平均値を使って、testデータを変換する
for col in cat_cols:
    # 学習データの統計量を計算する
    target_mean = train.groupby(col)[target_col].mean()
    
    # 欠損値を補完する
    test[col] = test[col].fillna('other')

    test[col+'_te'] = test[col].map(train.groupby(col)['取引価格（総額）_log'].mean())
    #testデータの欠損値にカテゴリ変数でグループ化した目的変数の値を入れる。
    test[col+'_te'] = test[col+'_te'].fillna(train.groupby(col)['取引価格（総額）_log'].mean().mean())


# trainデータとtestデータを結合
all_df = pd.concat([train, test], axis=0, ignore_index=True)

#print(all_df.isnull().sum())

#カテゴリ変換
#建築年を関数とif分を用いて、数値に変換
def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年' #'戦前'表記の場合は'昭和20年'に変換
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

all_df['建築年'] = all_df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))

#all_df['取引時点']の末尾'年'を削除する。
all_df['取引時点'] = all_df['取引時点'].str[:-3]

#all_df['取引時点']の末尾'年'を削除する。
#all_df['取引時点'].replace(all_df['取引時点'].iloc[1],'2013.4')
#若干ぬまったが、「Serise 文字変換」で検索するとすぐ出てきた。データ構造を意識して検索する！
all_df['取引時点'] = all_df['取引時点'].str.replace('年第','.')

### objectを数値型に変換する。

#最寄駅：距離（分）,面積（㎡）,建築年,取引時点
#'最寄駅：距離（分）'の文字列変換,'30分?60分'などは30~60分と予測して間の45分とする。
all_df['最寄駅：距離（分）'] = all_df['最寄駅：距離（分）'].replace('30分?60分','45')
all_df['最寄駅：距離（分）'] = all_df['最寄駅：距離（分）'].replace('1H?1H30','75')
all_df['最寄駅：距離（分）'] = all_df['最寄駅：距離（分）'].replace('2H?','120')
all_df['最寄駅：距離（分）'] = all_df['最寄駅：距離（分）'].replace('1H30?2H','105')
all_df['最寄駅：距離（分）'] = all_df['最寄駅：距離（分）'].astype(float)


#建築年
all_df['建築年'] = all_df['建築年'].astype(float)
#取引時点
all_df['取引時点'] = all_df['取引時点'].astype(float)

#label encoding：各文字列を本質的に意味のない数値型に変換する。
#['種類','最寄駅：名称','都道府県名','市区町村名','地区名','間取り','用途','都市計画','取引の事情等','建物の構造']
# 学習データに基づいて定義する

#カテゴリ変数のカラムを指定して、まとめて処理
def encode_categorical(df,cols):
    for col in cols:
        le = LabelEncoder()
        df[col] = pd.Series(le.fit_transform(df[col]))
    
    return df
all_df = encode_categorical(all_df,cols=["都道府県名","間取り","用途","都市計画","取引の事情等","建物の構造"])

#one-hot-encoding：カテゴリ変数が多くない特徴量はone-hotにする。
#今後の利用目的,改装,
# 変換するカテゴリ変数をリストに格納
cat_cols = ['今後の利用目的','改装']
all_df = pd.get_dummies(all_df,columns=cat_cols)

# 学習データとテストデータに再分割
train_x = all_df.iloc[:train_x.shape[0], :].reset_index(drop=True)
test = all_df.iloc[train_x.shape[0]:, :].reset_index(drop=True)

#print(all_df.info())

#欠損値しかないカラムを削除
nonnull_list = []
for col in all_df.columns:
  count = all_df[col].count()
  if count == 0:
    nonnull_list.append(col)
all_df = all_df.drop(nonnull_list, axis=1)    
print(all_df.info())

#市区町村名のカラムを削除
all_df = all_df.drop('市区町村名',axis=1)

#"種類"のカラムを削除
all_df = all_df.drop('種類',axis=1)

#"最寄駅：名称"のカラムを削除
all_df = all_df.drop('最寄駅：名称',axis=1)
#"地区名"のカラムを削除
all_df = all_df.drop('地区名',axis=1)

#面積と最寄駅：距離（分）にlogを取って、正規分布に近い形にする。
all_df['面積（㎡）_log'] = np.log(all_df['面積（㎡）'])
#面積と最寄駅：距離（分）にlogを取って、正規分布に近い形にする。
all_df['最寄駅：距離（分）_log'] = np.log(all_df['最寄駅：距離（分）'])

#特徴量が増えすぎたので、削除する。
#all_df = all_df.drop(['面積（㎡）','建築年','最寄駅：距離（分）','面積（㎡）_log','最寄駅：距離（分）_log','面積（㎡）_mean','最寄駅：距離（分）_mean'],axis=1)
#all_df = all_df.drop(['建築年','最寄駅：距離（分）'],axis=1)
all_df = all_df.drop(['面積（㎡）_log','最寄駅：距離（分）_log'],axis=1)

all_df = all_df.drop(['取引価格（総額）_log'],axis=1)
train_x = train_x.drop(['取引価格（総額）_log'],axis=1)
test = test.drop(['取引価格（総額）_log'],axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772094 entries, 0 to 772093
Data columns (total 27 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            772094 non-null  int64  
 1   種類            772094 non-null  object 
 2   市区町村コード       772094 non-null  int64  
 3   都道府県名         772094 non-null  int32  
 4   市区町村名         772094 non-null  object 
 5   地区名           772094 non-null  object 
 6   最寄駅：名称        772094 non-null  object 
 7   最寄駅：距離（分）     748889 non-null  float64
 8   間取り           772094 non-null  int32  
 9   面積（㎡）         772094 non-null  float64
 10  建築年           749595 non-null  float64
 11  建物の構造         772094 non-null  int32  
 12  用途            772094 non-null  int32  
 13  都市計画          772094 non-null  int32  
 14  建ぺい率（％）       746970 non-null  float64
 15  容積率（％）        746970 non-null  float64
 16  取引時点          772094 non-null  float64
 17  取引の事情等        772094 non-null  int32  
 18  取引価格

trainデータの数に対して、'地域','土地の形状','間口','延床面積（㎡）','前面道路：方位','前面道路：種類''前面道路：幅員（ｍ）'が全て欠損値

まとめたデータフレームをtrainとtestで分ける。  

In [3]:
# 学習データとテストデータに再分割
train_x = all_df.iloc[:train_x.shape[0], :].reset_index(drop=True)
test = all_df.iloc[train_x.shape[0]:, :].reset_index(drop=True)

## クロスバリデーション  
配列をランダムに抽出する。  
trainデータをhold-out法で分割する。

In [4]:
# train_test_split関数を用いてhold-out法で分割する
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y,
                                          test_size=0.25, random_state=3, shuffle=True)

## ベースラインモデルを「lightgbm」とする。

まずは前処理無しで予測した精度をベースラインとする。

# -----------------------------------
# lightgbmの実装

## バリデーションデータ作成  
今回のデータの注意点  
・データが県ごとのグループに分かれている。  
→ランダムにデータを取る必要がある。KFoldクラスで県ごとのグループを表す変数を分割して、それを元のデータを分割する
・欠損値が多い。

hold-out,クロスバリデーション,stratified k-hold,group k-fold,leave-one-outどれにするか検討する。  
・hold-outは県ごとで分けて予測する必要があるので無しかな  
・クロスバリデーションは使用候補。データ量を保ちつつ、バリデーションの評価に用いるデータを学習データ全体とできる。計算時間がネック。fold数は4,5かな。評価指標も今回はMAEなので各foldの平均とスコアは一致する。
・stratified k-fold1は分類タスクに使われることが多いから今回は微妙かな。  
・group k-foldでKFoldで県ごとに変数を分割する。  
・leave-one-outは学習データが少ない場合なので今回は無し。

In [6]:
# 訓練データ
lgb_train = lgb.Dataset(tr_x, tr_y)
# 評価データ
lgb_eval = lgb.Dataset(va_x, va_y, reference=lgb_train)

# パラメータの設定
parms = {
    'task': 'train', #トレーニング用
    'boosting': 'gbdt', #勾配ブースティング決定木
    'objective': 'regression_l1', #目的：MAE
    'metric': 'mae', #評価指標：正答率
    'num_iterations': 1000, #1000回学習
    'verbose': -1 #学習情報を非表示
}

# モデルの学習
model = lgb.train(parms,
                 #訓練データ
                 train_set=lgb_train,
                 # 評価データ
                 valid_sets=lgb_eval,
                 early_stopping_rounds=100)

# 結果の予測
va_pred = model.predict(va_x)

# 学習の実行、スコアの計算を行う
#model = Model()
#model.fit(tr_x, tr_y, va_x, va_y)
#va_pred = model.predict(va_x)
score = np.sqrt(mean_absolute_error(va_y, va_pred))
print(score)


[1]	valid_0's l1: 0.252352
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.236563
[3]	valid_0's l1: 0.222753
[4]	valid_0's l1: 0.210448
[5]	valid_0's l1: 0.199853
[6]	valid_0's l1: 0.18993
[7]	valid_0's l1: 0.1811
[8]	valid_0's l1: 0.173427
[9]	valid_0's l1: 0.16663
[10]	valid_0's l1: 0.160345
[11]	valid_0's l1: 0.154894
[12]	valid_0's l1: 0.150017
[13]	valid_0's l1: 0.145591
[14]	valid_0's l1: 0.141477
[15]	valid_0's l1: 0.138175
[16]	valid_0's l1: 0.134981
[17]	valid_0's l1: 0.131997
[18]	valid_0's l1: 0.129403
[19]	valid_0's l1: 0.127204
[20]	valid_0's l1: 0.125106
[21]	valid_0's l1: 0.123167
[22]	valid_0's l1: 0.121362
[23]	valid_0's l1: 0.119865
[24]	valid_0's l1: 0.118474
[25]	valid_0's l1: 0.117193
[26]	valid_0's l1: 0.116013
[27]	valid_0's l1: 0.114972
[28]	valid_0's l1: 0.113845
[29]	valid_0's l1: 0.112915
[30]	valid_0's l1: 0.112071
[31]	valid_0's l1: 0.111255
[32]	valid_0's l1: 0.110407
[33]	valid_0's l1: 0.109686
[34]	valid_0's l1: 0.108938

# ベースラインモデルの精度  
前処理は、カテゴリ変数変換のみ
### MAE 0.0871325  
となった。

# Publicスコアは 0.095668  

# 検証1

## 'ID'と'地域','土地の形状','間口','延床面積（㎡）','前面道路：方位','前面道路：種類','前面道路：幅員（ｍ）'(欠損値のみ)を削除して予測してみる。 
→精度下がった。  
## 面積と容積率を組み合わせて新たな特徴量を作る。
### 容積率（％）：建物の延べ面積（延床面積）の敷地面積に対する割合。これが大きいと高層マンションの可能性がたかい  
容積率×面積が大きいとそれほど、その部屋の家賃相場は上がると考えられる。  
建ぺい率：建物が占める土地面積/庭なども含めた敷地面積 ×100  
→精度下がった。  


# 検証4  
## 検証0のlightGBMでハイパーパラメータを調節して精度を上げる。

In [ ]:
SEED = 0

params = {
    'objective': 'mae', #目的関数を指定します。回帰問題の場合は、regressionを指定
    'metric': 'mae', # 評価指標を指定します。回帰問題の場合は、maeを指定
    'num_leaves': 10, #決定木の葉の数を指定します。この値を大きくすると、より複雑なモデルなる
    'max_depth': 7, #定木の深さの最大値を指定します。この値を大きくすると、より複雑なモデルになる
    "feature_fraction": 0.5, #各木で使用する特徴量の割合を指定します。過剰適合を抑制するために使用される
#バッチングサンプリング：学習データをバッチと呼ばれる小さなサイズのグループに分割し、その中からランダムにサンプリングする方法のこと
    'subsample_freq': 1, #バッチサンプリングの頻度を指定します。値が大きいほど、より多くのデータを使用される
    "bagging_fraction": 0.95, #バッグサンプリングで使用するサンプルの割合を指定します。過剰適合を抑制するために使用される
    'min_data_in_leaf': 100, #1つの葉に最低何個のデータポイントが必要かを指定します。
                           #この値が小さいほど、過剰適合のリスクが高まりますが、大きい場合はモデルが学習するのに時間がかかる可能性がある。
    'learning_rate': 0.1, #勾配ブースティングにおけるステップサイズを指定します。
                          #この値が小さいほど、モデルの収束が遅くなりますが、より正確な予測が可能になる
    "boosting": "gbdt", #使用するブースティングアルゴリズムを指定します。"gbdt"はGradient Boosting Decision Treeの略
    "lambda_l1": 0.1, #L1正則化の強さを指定します。L1正則化は、特徴量選択に役立つ
    "lambda_l2": 10, #L2正則化の強さを指定します。L2正則化は、過剰適合を抑制するために使用される
    "verbosity": -1, #出力レベルを設定します。-1は、ログを表示しないことを意味する。
    "random_state": 42, #ランダムシードを指定します。同じランダムシードを使用することで、実行ごとに同じ結果を得ることができる。
    "num_boost_round": 5000, #ブースティングの反復回数を指定します。
                              #この値が大きいほど、より多くの反復が行われますが、過剰適合のリスクが高まる可能性がある。
    "early_stopping_rounds": 100 #早期停止のための回数を指定します。モデルの性能が改善しなくなった場合、指定された回数後に学習が停止する。
}


精度向上

・"種類"データの削除。データが1種類のため。  
・集約特徴量を作成する。  
・単位面積あたりの価格を予測して後処理で面積をかけて最終予測にするモデル(diffモデル)を試す。 目的変数の変更。"面積"の対数をとって、本来予測したい値から面積_logを引いたものを予測するようにする。→「単位面積あたりの価格のlogをとったもの」を予測するモデルを学習するようになる。目的変数よりも正規分布に近くて予測しやすい。

In [ ]:
#集約特徴量を作成する。
#次元削減と、logを取ることにメリットが出てくる。
#集約のキーは"都道府県","地区名","最寄駅：名称"
#集約する特徴量は"建築年","面積（㎡）","最寄駅：距離（分）","面積（㎡）_log","最寄駅：距離（分）_log"

# 特定の列のヒストグラムを作成する
column_name = '面積（㎡）'  # ヒストグラムを作成する列の名前
all_df[column_name].hist()
all_df[column_name].hist(bins=20,range=(0,250))

# ヒストグラムのラベルやタイトルを設定する
plt.xlabel(column_name)
plt.ylabel('Frequency')
plt.title('Histogram of 面積（㎡）')

# ヒストグラムを表示する
plt.show()

In [ ]:
# 特定の列のヒストグラムを作成する
column_name = '最寄駅：距離（分）'  # ヒストグラムを作成する列の名前
all_df[column_name].hist()
all_df[column_name].hist(bins=20,range=(0,150))

# ヒストグラムのラベルやタイトルを設定する
plt.xlabel(column_name)
plt.ylabel('Frequency')
plt.title('Histogram of ' + column_name)

# ヒストグラムを表示する
plt.show()

In [ ]:
#面積と最寄駅：距離（分）にlogを取って、正規分布に近い形にする。
all_df['面積（㎡）_log'] = np.log(all_df['面積（㎡）'])

In [ ]:
#面積と最寄駅：距離（分）にlogを取って、正規分布に近い形にする。
all_df['最寄駅：距離（分）_log'] = np.log(all_df['最寄駅：距離（分）'])

# 検証12  
lightGBMで、optunaを用いてハイパーパラメータを調節してみる。

In [4]:
# train_test_split関数を用いてhold-out法で分割する
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y,
                                          test_size=0.25, random_state=3, shuffle=True)

In [ ]:
# LightGBMのデータセットを作成
train_data = lgb.Dataset(tr_x, label=tr_y)
valid_data = lgb.Dataset(va_x, label=va_y)

# ハイパーパラメータの最適化
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'feature_pre_filter': False,  # feature_pre_filterをfalseに設定する
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 150),
        'learning_rate': 0.1,
        'boosting': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'verbosity': -1,
        'random_state': 42,
        'num_boost_round': 5000,
        'early_stopping_rounds': 100,
        'objective': 'l1',
        'metric': 'mae'
    }
    
    # LightGBMのモデルを作成
    model = lgb.train(params, train_data, valid_sets=[train_data, valid_data], verbose_eval=False)
    
    # バリデーションデータに対する予測値を計算
    y_pred = model.predict(va_x, num_iteration=model.best_iteration)
    
    # 平均絶対誤差（MAE）を計算して返す
    return mean_absolute_error(va_y, y_pred)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100,timeout=10800) #timeoutの時間を3時間に設定。

# 最適なハイパーパラメータを出力
print(study.best_params)

{'num_leaves': 96, 'max_depth': 9, 'feature_fraction': 0.5200999026102292, 'subsample_freq': 2, 'bagging_fraction': 0.8562559055230834, 'min_data_in_leaf': 89, 'lambda_l1': 8.51128991582666, 'lambda_l2': 1.441516689466112e-05}

In [ ]:
SEED = 0

params = {
    'objective': 'l1', #目的関数を指定します。回帰問題の場合は、l1を指定
    'metric': 'mae', # 評価指標を指定します。回帰問題の場合は、maeを指定
    'num_leaves': 96, #決定木の葉の数を指定します。この値を大きくすると、より複雑なモデルなる
    'max_depth': 9, #定木の深さの最大値を指定します。この値を大きくすると、より複雑なモデルになる
    "feature_fraction": 0.5200999026102292, #各木で使用する特徴量の割合を指定します。過剰適合を抑制するために使用される
#バッチングサンプリング：学習データをバッチと呼ばれる小さなサイズのグループに分割し、その中からランダムにサンプリングする方法のこと
    'subsample_freq': 2, #バッチサンプリングの頻度を指定します。値が大きいほど、より多くのデータを使用される
    "bagging_fraction": 0.8562559055230834, #バッグサンプリングで使用するサンプルの割合を指定します。過剰適合を抑制するために使用される
    'min_data_in_leaf': 89, #1つの葉に最低何個のデータポイントが必要かを指定します。
                           #この値が小さいほど、過剰適合のリスクが高まりますが、大きい場合はモデルが学習するのに時間がかかる可能性がある。
    'learning_rate': 0.05, #勾配ブースティングにおけるステップサイズを指定します。
                          #この値が小さいほど、モデルの収束が遅くなりますが、より正確な予測が可能になる
    "boosting": "gbdt", #使用するブースティングアルゴリズムを指定します。"gbdt"はGradient Boosting Decision Treeの略
    "lambda_l1": 10, #L1正則化の強さを指定します。L1正則化は、特徴量選択に役立つ
    "lambda_l2": 1.441516689466112e-05, #L2正則化の強さを指定します。L2正則化は、過剰適合を抑制するために使用される
    "verbosity": -1, #出力レベルを設定します。-1は、ログを表示しないことを意味する。
    "random_state": 42, #ランダムシードを指定します。同じランダムシードを使用することで、実行ごとに同じ結果を得ることができる。
    "num_boost_round": 10000, #ブースティングの反復回数を指定します。
                              #この値が大きいほど、より多くの反復が行われますが、過剰適合のリスクが高まる可能性がある。
    "early_stopping_rounds": 80 #早期停止のための回数を指定します。モデルの性能が改善しなくなった場合、指定された回数後に学習が停止する。
}

train_data = lgb.Dataset(train_x, label=train_y)
val_data = lgb.Dataset(va_x, label=va_y)

model = lgb.train(
    params, #モデルのハイパーパラメータを指定する辞書形式の引数
    train_data, #トレーニング用のデータを指定するデータセットまたはnumpy配列
    #categorical_feature = cat_cols,
    valid_names = ['train', 'valid'], #トレーニング用のデータを指定するデータセットまたはnumpy配列
    valid_sets =[train_data, val_data],  #バリデーション用のデータを指定するデータセットまたはnumpy配列
    verbose_eval = 100, #訓練中に進行状況を出力する頻度を指定します。数値を指定すると、その間隔で進行状況が出力される
)

val_pred = model.predict(va_x, num_iteration=model.best_iteration)
score = mean_absolute_error(va_y, val_pred)

pred_df = pd.DataFrame(sorted(zip(va_x.index, val_pred, va_y)), columns=['index', 'predict', 'actual'])

feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(), train_x.columns)), columns=['importance', 'feature'])

print(f'score: {score:.4f}')

最終的には自分でパラメータを調節して精度が上がった。  
'objective'を'mae'ではなく'l1'にしたことで少し精度向上

精度はscore: 0.0657

In [7]:
# 結果の予測
test_pred = model.predict(test)


In [8]:
test_pred

array([7.63042569, 7.68515557, 7.67037361, ..., 7.45366859, 7.12424288,
       7.45294528])

In [9]:
sub = test.copy()
sub['取引価格（総額）_log'] = test_pred
sub.head(5)

,ID,市区町村コード,都道府県名,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,都市計画,建ぺい率（％）,容積率（％）,取引時点,取引の事情等,最寄駅：名称_te,地区名_te,今後の利用目的_その他,今後の利用目的_事務所,今後の利用目的_住宅,今後の利用目的_店舗,改装_改装済,改装_未改装,取引価格（総額）_log
0,1000057,1101,4,5.0,27,75.0,2007.0,4,23,0,80.0,600.0,2022.1,9,6.974706,6.94476,0,0,1,0,1,0,7.630426
1,1000077,1101,4,1.0,39,75.0,2016.0,4,23,0,80.0,600.0,2022.2,9,7.016224,6.94476,1,0,0,0,0,1,7.685156
2,1000081,1101,4,3.0,27,75.0,2012.0,4,23,0,80.0,600.0,2022.2,9,7.016224,6.94476,0,0,1,0,0,1,7.670374
3,1000128,1101,4,3.0,27,50.0,1981.0,4,23,0,80.0,400.0,2022.2,9,7.070514,6.94476,0,0,1,0,1,0,7.175543
4,1000129,1101,4,0.0,16,20.0,1977.0,8,23,0,80.0,400.0,2022.2,9,7.070514,6.94476,0,0,1,0,0,1,6.486682


In [10]:
# 提出ファイル
sub[['ID','取引価格（総額）_log']].to_csv('submit_16.csv', index=False)

In [11]:
# 提出ファイルをブラウザに保存する。
sub[['ID','取引価格（総額）_log']].to_csv(r"C:\Users\t-miy\OneDrive\デスクトップ\self-study\submit_16.csv", index=False)


# Publicデータが0.0730まで上がった!!  
## 4位/278位